In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [3]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('House_Reps_District.csv', dtype=col_types)

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). 

In [4]:
ZIP_to_CD_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially

In [5]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_COUNTY_062020_HUD.xlsx')
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)

# Sort the DataFrame by RES_RATIO in descending order
ZIP_to_County_df = ZIP_to_County_df.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_County_df = ZIP_to_County_df.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

In [6]:
ZIP_FEATURES = pd.merge(ZIP_to_CD_df, ZIP_to_County_df, on = "ZIP", how = "left")
ZIP_FEATURES

,ZIP,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY,COUNTY_RES_RATIO,COUNTY_BUS_RATIO,COUNTY_OTH_RATIO,COUNTY_TOT_RATIO
0,00501,3601,0.0,1.0,0.0,1.0,36103,0.000000,1.000000,0.000000,1.000000
1,00601,7200,1.0,1.0,1.0,1.0,72001,0.839242,0.800983,0.871166,0.837603
2,00602,7200,1.0,1.0,1.0,1.0,72003,1.000000,0.998801,1.000000,0.999919
3,00603,7200,1.0,1.0,1.0,1.0,72005,0.997152,0.998160,1.000000,0.997290
4,00604,7200,1.0,1.0,1.0,1.0,72005,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
45909,99925,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45910,99926,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45911,99927,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45912,99928,0200,0.0,0.0,1.0,1.0,02130,0.000000,0.000000,1.000000,1.000000


Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [7]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")
ZIP_FEATURES = ZIP_FEATURES.drop(columns=['FIPS_Code','Years_Served','District','CD_RES_RATIO',
                                                      'CD_BUS_RATIO','CD_OTH_RATIO','COUNTY_RES_RATIO',
                                                      'COUNTY_BUS_RATIO','COUNTY_OTH_RATIO'])

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS',
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)


ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,COUNTY_TOT_RATIO
0,00501,3601,1.0,"Zeldin, Lee M.",New York,Republican,36103,1.000000
1,00601,7200,1.0,Jenniffer González-Colón,Puerto Rico,Republican,72001,0.837603
2,00602,7200,1.0,Jenniffer González-Colón,Puerto Rico,Republican,72003,0.999919
3,00603,7200,1.0,Jenniffer González-Colón,Puerto Rico,Republican,72005,0.997290
4,00604,7200,1.0,Jenniffer González-Colón,Puerto Rico,Republican,72005,1.000000
...,...,...,...,...,...,...,...,...
48291,99925,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
48292,99926,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
48293,99927,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
48294,99928,0200,1.0,"Young, Don",Alaska,Republican,02130,1.000000


In [15]:
ZIP_FEATURES.to_csv('ZIP_Features.csv')

Pulling from their API to see if I can extract any more years worth of data \
It looks like it's only cumulative.

In [13]:
import requests
import json

# API endpoint
url = "https://data.cdc.gov/resource/kn79-hsxy.json"

# Making a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    
    # Extracting all 'data_as_of' values
    data_as_of_values = [item['data_as_of'] for item in data if 'data_as_of' in item]

    # Getting unique values by converting the list to a set
    unique_data_as_of_values = set(data_as_of_values)

    # Printing unique 'data_as_of' values
    print("Unique 'data_as_of' values:")
    for value in unique_data_as_of_values:
        print(value)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Unique 'data_as_of' values:
2023-06-28T00:00:00.000


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits)

In [8]:
col_types = {
    'FIPS County Code': str
}

CDC_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/Provisional_COVID-19_Death_Counts_in_the_United_States_by_County.csv')
CDC_deaths['FIPS County Code'] = CDC_deaths['FIPS County Code'].astype(str).str.zfill(5)

CDC_deaths = DCD_deaths.drop(columns = [''])
CDC_deaths.head()

,Date as of,Start Date,End Date,State,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19,Deaths from All Causes,Footnote
0,06/28/2023,01/01/2020,06/24/2023,AK,Aleutians East Borough,02013,Noncore,NaN,25,One or more data cells have counts between 1-9...
1,06/28/2023,01/01/2020,06/24/2023,AK,Anchorage Municipality,02020,Medium metro,775.0,8869,NaN
2,06/28/2023,01/01/2020,06/24/2023,AK,Bethel Census Area,02050,Noncore,40.0,395,NaN
3,06/28/2023,01/01/2020,06/24/2023,AK,Denali Borough,02068,Noncore,NaN,31,One or more data cells have counts between 1-9...
4,06/28/2023,01/01/2020,06/24/2023,AK,Dillingham Census Area,02070,Noncore,NaN,115,One or more data cells have counts between 1-9...
